***GENERATED CODE FOR decisiontree PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '251', 'mean': '', 'stddev': '', 'min': '0029A597-0CAB-4B2F-9018-CED664D22BB4', 'max': 'FDD00F9A-2CEB-4913-AE64-0287450C707B', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateofService'}, {'feature_label': 'DateofService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateofService')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfEntry'}, {'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'TransactionDate', 'transformation_label': 'Extract Date'}], 'feature': 'TransactionDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'TransactionDate'}, {'feature_label': 'TransactionDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('TransactionDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'}], 'feature': 'ReasonDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ReasonDate'}, {'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'CloseDate', 'transformation_label': 'Extract Date'}], 'feature': 'CloseDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'CloseDate'}, {'feature_label': 'CloseDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('CloseDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '92200.12', 'stddev': '19732.39', 'min': '11042', 'max': 'J1040', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '9.46', 'stddev': '24.76', 'min': '0000', 'max': 'Q8', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': 'A41.9', 'max': 'Z85.841', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': 'B96.29', 'max': 'Z99.2', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'}], 'feature': 'Reasoncode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '152.26', 'stddev': '103.66', 'min': '109', 'max': 'N95', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reasoncode'}, {'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reasoncode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': 'Authorization', 'max': 'Requires Review', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonCategory'}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': 'Alert: Consult our contractual agreement for restrictions/billing/payment information related to these charges.', 'max': 'This service/equipment/drug is not covered under the patient’s current benefit plan', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDescription'}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': 'Cardiovascular', 'max': 'TeleMedicine Visit', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': 'Cardiac Surgery', 'max': 'Vascular Surgery', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': '04890B9B-DCEE-4476-80D4-DBD873535830', 'max': 'FED3AED0-2A50-445D-86B3-3A2902878AC4', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': '1D3F7401-F2E8-4272-A62A-051B8BCDC8F6', 'max': 'DEBDDFBC-6029-48BD-B892-E1A0963E4BA7', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature_label': 'PayerId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondaryPayerID', 'transformation_label': 'String Indexer'}], 'feature': 'SecondaryPayerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': '027330C1-DCB1-4CCE-8D3D-6D4DC325CFAF', 'max': 'FC574DD7-0F1E-4350-A8F8-A5F7CB490D60', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondaryPayerID'}, {'feature_label': 'SecondaryPayerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondaryPayerID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TertiaryPayerID', 'transformation_label': 'String Indexer'}], 'feature': 'TertiaryPayerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': '027330C1-DCB1-4CCE-8D3D-6D4DC325CFAF', 'max': 'FC95A7E1-E213-4010-9EAE-6C472C4C2EB7', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TertiaryPayerID'}, {'feature_label': 'TertiaryPayerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TertiaryPayerID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


def gradientboostingclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    Decision_tree_classifier = DecisionTreeClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    Decision_tree_classifier.fit(X_train, y_train)
    display(" Accuracy of Model : %s" %
            Decision_tree_classifier.score(X_test, y_test))

    data = {'model': Decision_tree_classifier,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run decisiontreeHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'DenialWorkflow', 'optionalDB': 'DenialWorkflow', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT * FROM\r\n\r\n(\r\n\r\n                                                SELECT DISTINCT\r\n\r\n                                                cd.ChargeDetailID,\r\n\r\n                                                cd.DateofService,\r\n\r\n                                                cd.DateOfEntry,\r\n\r\n                                                cd.Amount,\r\n\r\n                                                t.TransactionDate,\r\n\r\n                                                tr.ReasonDate,\r\n\r\n                                                t.CloseDate,\r\n\r\n                                                cd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\n\r\n                                                cptc.CPTCode,\r\n\r\n                                                CASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\r\n                                                dc.DiagnosisCode as pdiagnosis1,\r\n\r\n                                                dc2.DiagnosisCode as pdiagnosis2,\r\n\r\n                                                IsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END),\r\n\r\n                                                Isauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\r\n                                                PredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\r\n                                                MedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\r\n                                                FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\r\n                                                DATEDIFF(day, t.TransactionDate, tr.ReasonDate) as days_to_denial,\r\n\r\n                                                trc.Reasoncode,\r\n\r\n                                                trc.ReasonCategory,\r\n\r\n                                                trc.ReasonDescription,\r\n\r\n                                                d.DepartmentCategory,\r\n\r\n                                                s.SpecialityCategory,\r\n\r\n                                                pp.ProviderID,\r\n\r\n                                                t.PayerId,t.SecondaryPayerID,t.TertiaryPayerID,\r\n\r\n                                                label = 0\r\n\r\n                                               \r\n\r\n                                                from ChargeDetail cd\r\n\r\n                                                inner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n\r\n                                                left outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = e.PayerID \r\n\r\n                                                inner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\r\n                                                inner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\n\r\n                                                inner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\r\n                                                inner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\r\n                                                inner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\r\n                                                inner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\r\n                                                inner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\r\n                                                inner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\r\n                                                inner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\r\n                                                inner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\r\n                                                inner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\r\n                                                WHERE\r\n\r\n                                                cd.DateOfService between '1/01/2020' and '12/31/2020'\r\n\r\n                                                and cd.Amount >.01 and t.CloseDate is not null\r\n\r\n                                                and trc.reasonType = 'Denial' \r\n\r\n                                                and tt.TransactionTypeCategory = 'Charges'\r\n\r\n                                                and trc.ReasonCode IN ('19','20','21','22','24','26','27','28','30','31','32')\r\n\r\n \r\n\r\nUNION\r\n\r\n \r\n\r\nSELECT DISTINCT\r\n\r\n \r\n\r\n                cd.ChargeDetailID,\r\n\r\n                cd.DateofService,\r\n\r\n                cd.DateOfEntry,\r\n\r\n                cd.Amount,\r\n\r\n                t.TransactionDate,\r\n\r\n                tr.ReasonDate,\r\n\r\n                t.CloseDate,\r\n\r\n                cd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\n\r\n                cptc.CPTCode,\r\n\r\n                CASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\r\n                dc.DiagnosisCode as pdiagnosis1,\r\n\r\n                dc2.DiagnosisCode as pdiagnosis2,\r\n\r\n                IsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END),\r\n\r\n                Isauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\r\n                PredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\r\n                MedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\r\n                FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\r\n                DATEDIFF(day, t.TransactionDate, tr.ReasonDate) as days_to_denial,\r\n\r\n                trc.Reasoncode,\r\n\r\n                trc.ReasonCategory,\r\n\r\n                trc.ReasonDescription,\r\n\r\n                d.DepartmentCategory,\r\n\r\n                s.SpecialityCategory,\r\n\r\n                pp.ProviderID,\r\n\r\n                t.PayerId,t.SecondaryPayerID,t.TertiaryPayerID,\r\n\r\n                label = 1\r\n\r\n               \r\n\r\n                from ChargeDetail cd\r\n\r\n                inner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n\r\n                left outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = e.PayerID \r\n\r\n                inner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\r\n                inner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\n\r\n                inner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\r\n                inner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\r\n                inner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\r\n                inner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\r\n                inner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\r\n                inner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\r\n                inner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\r\n                inner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\r\n                inner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\r\n                WHERE\r\n\r\n                cd.DateOfService between '1/01/2020' and '12/31/2020'\r\n\r\n                and cd.Amount >.01 and t.CloseDate is not null\r\n\r\n                and trc.reasonType = 'Denial' \r\n\r\n                and tt.TransactionTypeCategory = 'Charges'\r\n\r\n                and trc.ReasonCode NOT IN ('19','20','21','22','24','26','27','28','30','31','32')\r\n\r\n \r\n\r\n) as DND", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run decisiontreeHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "0029A597-0CAB-4B2F-9018-CED664D22BB4", "max": "FDD00F9A-2CEB-4913-AE64-0287450C707B", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "DateofService", "transformation_label": "Extract Date"}], "feature": "DateofService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateofService"}, {"transformationsData": [{"feature_label": "DateOfEntry", "transformation_label": "Extract Date"}], "feature": "DateOfEntry", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfEntry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "251", "mean": "197.33", "stddev": "183.92", "min": "2.31", "max": "1335.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "TransactionDate", "transformation_label": "Extract Date"}], "feature": "TransactionDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "TransactionDate"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "Extract Date"}], "feature": "ReasonDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ReasonDate"}, {"transformationsData": [{"feature_label": "CloseDate", "transformation_label": "Extract Date"}], "feature": "CloseDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "CloseDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "251", "mean": "0.25", "stddev": "3.09", "min": "-21.84", "max": "31.62", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PatientBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "251", "mean": "8.9", "stddev": "42.8", "min": "-46.4", "max": "353.12", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InsuranceBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "251", "mean": "3.46", "stddev": "20.27", "min": "0.0", "max": "218.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "92200.12", "stddev": "19732.39", "min": "11042", "max": "J1040", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "9.46", "stddev": "24.76", "min": "0000", "max": "Q8", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "A41.9", "max": "Z85.841", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "B96.29", "max": "Z99.2", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsBillable", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "251", "mean": "0.86", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "IsBillable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Isauthorized", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "251", "mean": "0.25", "stddev": "0.43", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Isauthorized"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredeterminationIndicator", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "251", "mean": "0.22", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "PredeterminationIndicator"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MedNecessityRequiredIndicator", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "251", "mean": "0.21", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "MedNecessityRequiredIndic..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "251", "mean": "73.94", "stddev": "13.62", "min": "25", "max": "99", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_to_denial", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "251", "mean": "46.88", "stddev": "49.65", "min": "4", "max": "268", "missing": "0"}, "updatedLabel": "days_to_denial"}, {"transformationsData": [{"feature_label": "Reasoncode", "transformation_label": "String Indexer"}], "feature": "Reasoncode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "152.26", "stddev": "103.66", "min": "109", "max": "N95", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reasoncode"}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "Authorization", "max": "Requires Review", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonCategory"}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "Alert: Consult our contractual agreement for restrictions/billing/payment information related to these charges.", "max": "This service/equipment/drug is not covered under the patient\u2019s current benefit plan", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDescription"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "Cardiovascular", "max": "TeleMedicine Visit", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "Cardiac Surgery", "max": "Vascular Surgery", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"feature_label": "ProviderID", "transformation_label": "String Indexer"}], "feature": "ProviderID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "04890B9B-DCEE-4476-80D4-DBD873535830", "max": "FED3AED0-2A50-445D-86B3-3A2902878AC4", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderID"}, {"transformationsData": [{"feature_label": "PayerId", "transformation_label": "String Indexer"}], "feature": "PayerId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "1D3F7401-F2E8-4272-A62A-051B8BCDC8F6", "max": "DEBDDFBC-6029-48BD-B892-E1A0963E4BA7", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerId"}, {"transformationsData": [{"feature_label": "SecondaryPayerID", "transformation_label": "String Indexer"}], "feature": "SecondaryPayerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "027330C1-DCB1-4CCE-8D3D-6D4DC325CFAF", "max": "FC574DD7-0F1E-4350-A8F8-A5F7CB490D60", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondaryPayerID"}, {"transformationsData": [{"feature_label": "TertiaryPayerID", "transformation_label": "String Indexer"}], "feature": "TertiaryPayerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "027330C1-DCB1-4CCE-8D3D-6D4DC325CFAF", "max": "FC95A7E1-E213-4010-9EAE-6C472C4C2EB7", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TertiaryPayerID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "label", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "251", "mean": "0.86", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "label"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run decisiontreeHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=DecisionTreeClassifier(autofe, ["ChargeDetailID_stringindexer", "Amount", "PatientBal", "InsuranceBal", "PrimaryInsuranceBal", "CPTCode_stringindexer", "CptModifier_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "IsBillable", "Isauthorized", "PredeterminationIndicator", "MedNecessityRequiredIndicator", "Age", "Gender_stringindexer", "days_to_denial", "Reasoncode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "ProviderID_stringindexer", "PayerId_stringindexer", "SecondaryPayerID_stringindexer", "TertiaryPayerID_stringindexer", "DateofService_dayofmonth", "DateofService_month", "DateofService_year", "DateOfEntry_dayofmonth", "DateOfEntry_month", "DateOfEntry_year", "TransactionDate_dayofmonth", "TransactionDate_month", "TransactionDate_year", "ReasonDate_dayofmonth", "ReasonDate_month", "ReasonDate_year", "CloseDate_dayofmonth", "CloseDate_month", "CloseDate_year"], "label")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
